# Déchiffrement de la syntaxe Prolog

In [1]:
type var = Id of string * int

type table = Empty | NonEmpty of term * table | TVar of var
and term = Var of var | Predicate of string * (term list) | Table of table

type clause = Clause of term * (term list);;

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



type var = Id of string * int


type table = Empty | NonEmpty of term * table | TVar of var
and term = Var of var | Predicate of string * term list | Table of table


type clause = Clause of term * term list


## Le nouveau Parser :

In [2]:
type structure = S of string | V of var | L of table | T of term | C of clause 
                | W of clause list | P of string | TL of term list

type structure =
    S of string
  | V of var
  | L of table
  | T of term
  | C of clause
  | W of clause list
  | P of string
  | TL of term list


In [3]:
let to_list str = List.init (String.length str) (String.get str);;
let rec remove_spaces = function
| [] -> []
| (' ' | '\t' | '\n')::l -> remove_spaces l
| a::l -> a::(remove_spaces l);;
let of_char c = String.make 1 c;;

val to_list : string -> char list = <fun>


val remove_spaces : char list -> char list = <fun>


val of_char : char -> string = <fun>


In [4]:
let ( <+> ) ra rb = fun l -> (ra l) @ (rb l)

let chain y l = List.map (fun (a, b) -> (a, y@b)) l

let ( *~* ) ra rb = 
let chain y l = List.map (fun (a, b) -> (a, y@b)) l in
                    fun l -> List.concat (
                        List.map (fun (x, y) -> chain y (rb x)) (ra l)
                    )
                    
let ( >~> ) ra f = fun l -> List.map (fun (a, b) -> (a, f b)) (ra l)

let skip = fun l -> []
                    
let fix aux = let rec g x = aux g x in g

val ( <+> ) : ('a -> 'b list) -> ('a -> 'b list) -> 'a -> 'b list = <fun>


val chain : 'a list -> ('b * 'a list) list -> ('b * 'a list) list = <fun>


val ( *~* ) :
  ('a -> ('b * 'c list) list) ->
  ('b -> ('d * 'c list) list) -> 'a -> ('d * 'c list) list = <fun>


val ( >~> ) : ('a -> ('b * 'c) list) -> ('c -> 'd) -> 'a -> ('b * 'd) list =
  <fun>


val skip : 'a -> 'b list = <fun>


val fix : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


In [5]:
let rlow = function
| ('a'..'z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rup = function
| ('A'..'Z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rsym s = function
| h::t when s = h -> [t, [S (of_char s)]]
| _ -> []

let rchar = function
| ('a'..'z' | 'A'..'Z' | '0'..'9' | '_' as c)::t -> [t, [S (of_char c)]]
| _ -> []

let rmot =
let aux self = rchar 
            <+> rchar *~* self >~> (function | [S s; S s'] -> [S (s^s')] | l -> l)
in fix aux

let variable = 
    rup >~> (function | [S s] -> [V (Id (s,0) )] | l -> l)
<+> rup *~* rmot >~> (function | [S s; S s'] -> [V (Id ((s^s'),0) )] | l -> l)

let predicate = rlow >~> (function | [S s] -> [P s] | l -> l)
                <+> rlow *~* rmot >~> (function | [S s; S s'] -> [P (s^s')] | l -> l)

let rec table_from_list tl t = match tl with
| h::q -> NonEmpty (h, table_from_list q t)
| [] -> t

let rec term l = (
    variable >~> (function | [V s] -> [T (Var s)] | l -> l)
<+> predicate >~> (function | [P s] -> [T (Predicate (s,[]))] | l -> l)
<+> predicate *~* (rsym '(' >~> skip) *~* termlist *~* (rsym ')' >~> skip) 
        >~> (function | [P s; TL tl] -> [T (Predicate (s, tl))] | l -> l)
<+> table >~> (function | [L t] -> [T (Table t)] | l -> l)
) l
and termlist l = (
term >~> (function | [T t] -> [TL [t]] | l -> l)
<+> term *~* (rsym ',' >~> skip) *~* termlist >~> (function | [T t; TL tl] -> [TL (t::tl)] | l -> l)
) l
and table l = (
rsym '[' *~* rsym ']' >~> (function | [S "["; S "]"] -> [L Empty] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl] -> [L (table_from_list tl Empty)] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym '|' >~> skip) *~* table *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl; L t] -> [L (table_from_list tl t)] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym '|' >~> skip) *~* variable *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl; V v] -> [L (table_from_list tl (TVar v))] | l -> l)
) l


let clause = term *~* (rsym '.' >~> skip) >~> (function | [T t] -> [C (Clause (t,[]))] | l -> l)
            <+> term *~* (rsym ':' *~* rsym '-' >~> skip) *~* termlist *~* (rsym '.' >~> skip)
                    >~> (function | [T t; TL tl] -> [C (Clause (t,tl))] | l -> l)
let programme =
let aux self = clause >~> (function | [C c] -> [W [c]] | l -> l)
            <+> clause *~* self >~> (function | [C c; W cl] -> [W (c::cl)]| l -> l)
in fix aux

let check r l = 
let aux = function
| [], result -> Some result
| _ -> None
in match List.filter_map aux (r l) with
| [] -> failwith "Parsing failed"
| [a] -> a
| _ -> failwith "Ambiguous grammar";;

let parse_program s = match (s |> to_list |> remove_spaces |> (check programme)) with
| [W w] -> w
| _ -> failwith "input is not a program"

let parse_term s = match (s |> to_list |> remove_spaces |> (check term)) with
| [T t] -> t
| _ -> failwith "input is not a term"

let parse_clause c = match (c |> to_list |> remove_spaces |> (check clause)) with
| [C a] -> a
| _ -> failwith "input is not a clause"

let parse_termlist str = match (str |> to_list |> remove_spaces |> (check termlist)) with
| [TL tl] -> tl
| _ -> failwith "input is not a termlist"

val rlow : char list -> (char list * structure list) list = <fun>


val rup : char list -> (char list * structure list) list = <fun>


val rsym : char -> char list -> (char list * structure list) list = <fun>


val rchar : char list -> (char list * structure list) list = <fun>


val rmot : char list -> (char list * structure list) list = <fun>


...

val check : ('a -> ('b list * 'c) list) -> 'a -> 'c = <fun>


val parse_program : string -> clause list = <fun>


val parse_term : string -> term = <fun>


val parse_clause : string -> clause = <fun>


val parse_termlist : string -> term list = <fun>


# Fonctions de base

In [6]:
module TermSet = Set.Make(struct
   type t = term * term
   let compare = compare
end)

module VarMap = Map.Make(struct
   type t = var
   let compare = compare
end)

module TermSet :
  sig
    type elt = term * term
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val disjoint : t -> t -> bool
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option


module VarMap :
  sig
    type key = var
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a t -> (key *

In [7]:
(* Transforme un terme en une chaîne de caractères *)
let rec string_of_term = function
| Predicate (p, []) -> p
| Predicate (p, l) -> p ^ "(" ^ (String.concat ", " (List.map string_of_term l)) ^ ")"
| Var (Id (s, _)) -> s
| Table (Empty) -> "[]"
| Table (TVar (Id (s,_))) -> s
| Table (NonEmpty (t, l)) -> 
let rec aux = function
| Empty -> ""
| TVar (Id (s, _)) -> "|"^s
| NonEmpty (t, l) -> ", "^(string_of_term t)^(aux l)
in "["^(string_of_term t)^(aux l)^"]"

(* DEBUG : affiche un liste de couple de termes *)
let print_eqs eqs = 
TermSet.iter (fun (t1,t2) -> Format.printf "%s <-> %s\n%!" (string_of_term t1) (string_of_term t2)) eqs

val string_of_term : term -> string = <fun>


val print_eqs : TermSet.t -> unit = <fun>


In [8]:
(* Applique une substitution sur un terme *)
(* Attention, on ne peut remplacer une variable qui représente un tableau que par un tableau *)
let rec replace_var_in_term var new_term term = match term with
| Var v -> if v = var then new_term else Var v
| Predicate (p, l) -> Predicate (p, List.map (replace_var_in_term var new_term) l)
| Table tbl ->
let rec aux = function
| Empty -> Empty
| TVar v -> if v=var then
(match new_term with | Table nt -> nt 
| _ -> failwith "Try to replace a table by a term that is not a table")
else TVar v
| NonEmpty (head, tail) -> NonEmpty (replace_var_in_term var new_term head, aux tail)
in Table (aux tbl)

let replace_var_in_eqs var new_term eqs = 
TermSet.map (fun (a, b) -> replace_var_in_term var new_term a, replace_var_in_term var new_term b) eqs

let rec var_in_term var = function
| Var v -> if v=var then true else false
| Predicate (p, l) -> List.mem true (List.map (var_in_term var) l)
| Table t -> let rec aux = function
| Empty -> false
| NonEmpty (head, tail) -> var_in_term var head || aux tail
| TVar v -> if v=var then true else false
in aux t

let var_in_eqs var eqs = 
TermSet.exists (function (a, b) -> var_in_term var a || var_in_term var b) eqs;;


(* Recherche les termes qui sont des variables récursivement dans une liste de termes *)
let rec find_vars_in_termlist tl = 
let rec find_vars_in_term = function
| Var v -> [Var v]
| Predicate (_, l) -> find_vars_in_termlist l
| Table t ->
let rec aux = function
| Empty -> []
| TVar v -> [Table (TVar v)]
| NonEmpty (head, tail) -> (find_vars_in_term head)@(aux tail)
in aux t
in List.sort_uniq compare (List.concat (List.map find_vars_in_term tl));;

(* Renvoie la liste des variables représentant un tableau à l'intérieur d'un terme *)
let rec find_tvars_in_term = function
| Var _ -> []
| Predicate (_, l) -> List.concat (List.map find_tvars_in_term l)
| Table t -> 
let rec aux = function
| Empty -> []
| NonEmpty (head, tail) -> (find_tvars_in_term head) @ (aux tail)
| TVar v -> [v]
in aux t

(* Renvoie la liste des variables représentant un tableau à l'intérieur d'une liste de termes *)
let rec find_tvars_in_termlist tl = List.concat (List.map find_tvars_in_term tl)

(* Renvoie la liste des variables représentant un tableau à l'intérieur d'une clause *)
let find_tvars_in_clause (Clause (t, tl)) = (find_tvars_in_term t) @ (find_tvars_in_termlist tl)

val replace_var_in_term : var -> term -> term -> term = <fun>


val replace_var_in_eqs : var -> term -> TermSet.t -> TermSet.t = <fun>


val var_in_term : var -> term -> bool = <fun>


val var_in_eqs : var -> TermSet.t -> bool = <fun>


val find_vars_in_termlist : term list -> term list = <fun>


val find_tvars_in_term : term -> var list = <fun>


val find_tvars_in_termlist : term list -> var list = <fun>


val find_tvars_in_clause : clause -> var list = <fun>


# Vérification des types

In [9]:
let type_check_term term =
List.fold_left (fun t v -> replace_var_in_term (v) (Table (TVar v)) t) term (find_tvars_in_term term)

let type_check_cl cl = 
let l = find_tvars_in_clause cl in
List.fold_left (fun cl v -> 
let aux = replace_var_in_term (v) (Table (TVar v))
and Clause (left, right) = cl in
Clause (aux left, List.map aux right)
) cl l

let read_term str = type_check_term (parse_term str)
let read_program str = List.map type_check_cl (parse_program str)
let read_clause str = type_check_cl (parse_clause str)
let read_termlist str = List.map type_check_term (parse_termlist str)

val type_check_term : term -> term = <fun>


val type_check_cl : clause -> clause = <fun>


val read_term : string -> term = <fun>


val read_program : string -> clause list = <fun>


val read_clause : string -> clause = <fun>


val read_termlist : string -> term list = <fun>


# Algorithme d'unification

In [10]:
(* E est un ensemble couples de termes *)
(* transforme E en un équivalent sous forme résolue (type option, None si impossible) *)
type equation_set = {mutable data : TermSet.t; mutable correct : bool}

let rec solve e =
let eqs = {data = e; correct = true} in

let aux = function
| Predicate (f, lf), Predicate (g, lg) when f = g && List.length lf = List.length lg ->
    List.iter2 (fun a b -> eqs.data <- TermSet.add (a,b) eqs.data) lf lg;
    eqs.data <- TermSet.remove (Predicate (f, lf), Predicate (g, lg)) eqs.data;

| Predicate (_, _), Predicate (_, _) -> 
    eqs.correct <- false;

| Var x, Var y when x = y ->
    eqs.data <- TermSet.remove (Var x, Var y) eqs.data

| Predicate (a, la), Var x ->
    eqs.data <- TermSet.remove (Predicate (a, la), Var x) eqs.data;
    eqs.data <- TermSet.add    (Var x, Predicate (a, la)) eqs.data;

| Table t, Var x ->
    eqs.data <- TermSet.remove (Table t, Var x) eqs.data;
    eqs.data <- TermSet.add    (Var x, Table t) eqs.data;

| Var w, t when var_in_eqs w (TermSet.remove (Var w, t) eqs.data) ->
    if var_in_term w t then eqs.correct <- false
    else (
    eqs.data <- TermSet.remove (Var w, t) eqs.data;
    eqs.data <- replace_var_in_eqs w t eqs.data;
    eqs.data <- TermSet.add (Var w, t) eqs.data;
    )

| Var v, t ->
    ()

| Predicate (_, _), Table _ -> 
    eqs.correct <- false

| Table _, Predicate (_, _) -> 
    eqs.correct <- false

| Table t1, Table t2 -> (
    match t1, t2 with 
    | Empty, Empty ->
        eqs.data <- TermSet.remove (Table Empty, Table Empty) eqs.data;

    | Empty, NonEmpty _ ->
        eqs.correct <- false;

    | NonEmpty _, Empty -> 
        eqs.correct <- false;

    | (NonEmpty (head1, tail1) as t1), (NonEmpty (head2, tail2) as t2) ->
        eqs.data <- TermSet.remove (Table t1, Table t2) eqs.data;
        eqs.data <- TermSet.add (head1, head2) eqs.data;
        eqs.data <- TermSet.add (Table tail1, Table tail2) eqs.data;

    | (NonEmpty (_, _) | Empty as t), TVar a ->
        eqs.data <- TermSet.remove (Table t, Table (TVar a)) eqs.data;
        eqs.data <- TermSet.add (Table (TVar a), Table t) eqs.data;

    | TVar a, TVar b when a=b ->
        eqs.data <- TermSet.remove (Table (TVar a), Table (TVar b)) eqs.data;

    | TVar w, table when var_in_eqs w (TermSet.remove (Table (TVar w), Table table) eqs.data) ->
        if var_in_term w (Table table) then eqs.correct <- false
        else eqs.data <- replace_var_in_eqs w (Table table) eqs.data;

    | TVar _ , _ -> 
        ()
    )
in
let old = ref TermSet.empty in 
while eqs.data <> !old && eqs.correct do
old := eqs.data;
TermSet.iter (fun c -> if eqs.data <> !old then () else aux c) eqs.data;
done;

if eqs.correct then Some eqs.data else None

type equation_set = { mutable data : TermSet.t; mutable correct : bool; }


val solve : TermSet.t -> TermSet.t option = <fun>


In [11]:
(* Prend 2 terme et renvoie l'unifieur le plus général s'il existe, None sinon. 
Attention, aucune variable ne doit être présente dans 
(r) la requête et dans (c) la tête de la clause *)

let rec mgu r c = let f = function
    | Var x, t -> x, t
    | Predicate (_, _), _ -> failwith "Should not happen."
    | Table (TVar r), t -> r, t
    | Table (Empty | NonEmpty _), _ -> failwith "Should not happen."
in match solve (TermSet.singleton (r, c)) with
| None -> None
| Some e -> Some (e |> TermSet.to_seq |> Seq.map f |> VarMap.of_seq);;

val mgu : term -> term -> term VarMap.t option = <fun>


In [12]:
(* La fonction test_unification a pour but de tester mgu *)
let test_unification str_r str_c = Format.printf "Unification de %s et de %s :\n%!" str_r str_c;
match mgu (str_r |> read_term) (str_c |> read_term) with
| Some t -> VarMap.iter 
    (fun (Id (s, a)) term -> Format.printf "%s <- %s\n%!" s (string_of_term term)) t
| None -> Format.printf "No unification\n%!"
;;

test_unification "etudiant_de(E, pierre)" "etudiant_de(F,P)"; (* F/E et P/pierre *)

test_unification "etudiant_de(F,P)" "etudiant_de(E, pierre)"; (* E/F et P/pierre *)

test_unification "f(X,g(Y))" "f(g(Z),Z)"; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "f(g(Z),Z)" "f(X,g(Y))"; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "[a, a, b, c]" "[A | B]";

test_unification "[a, a, b, c]" "[A | A]";

test_unification "[[a, b, c], a, b, c]" "[A | A]"

val test_unification : string -> string -> unit = <fun>


Unification de etudiant_de(E, pierre) et de etudiant_de(F,P) :
E <- F
P <- pierre
Unification de etudiant_de(F,P) et de etudiant_de(E, pierre) :
F <- E
P <- pierre
Unification de f(X,g(Y)) et de f(g(Z),Z) :
X <- g(g(Y))
Z <- g(Y)
Unification de f(g(Z),Z) et de f(X,g(Y)) :
X <- g(g(Y))
Z <- g(Y)
Unification de [a, a, b, c] et de [A | B] :
A <- a
B <- [a, b, c]
Unification de [a, a, b, c] et de [A | A] :


- : unit = ()


# Algorithme de backtracking

In [13]:
(* Applique une substitution *)
let apply_subst_on_term = VarMap.fold replace_var_in_term;;

(* Applique une substitution sur une liste de termes *)
let apply_subst_on_termlist uni = apply_subst_on_term uni |> List.map;;

No unification
Unification de [[a, b, c], a, b, c] et de [A | A] :
A <- [a, b, c]


val apply_subst_on_term : term VarMap.t -> term -> term = <fun>


val apply_subst_on_termlist : term VarMap.t -> term list -> term list = <fun>


In [14]:
(* Prend un entier et une clause, renvoie la clause avec les variables renommées à l'entier *)
let rename n (Clause (t1, tl)) = 
let rec f = function 
| Var (Id (str, _)) -> Var (Id (str, n))
| Predicate (atm, l) -> Predicate (atm, List.map f l)
| Table t -> 
let rec aux = function
| Empty -> Empty
| TVar (Id (str, _)) -> TVar (Id (str, n))
| NonEmpty (head, tail) -> NonEmpty (f head, aux tail)
in Table (aux t)
in Clause (f t1, List.map f tl);;

val rename : int -> clause -> clause = <fun>


In [15]:
type 'a tree = Leaf of 'a | Node of (('a tree) Lazy.t) list;;

(* La fonction de recherche, renvoie l'arbre des solutions *)
let rec sld_tree world req subs n = match req with
| [] -> Leaf subs 
| head_request_term::other_request_terms -> 
    Node (List.filter_map (fun c -> let Clause (left_member, right_member) = rename n c in 
    (match mgu head_request_term left_member with
        | None -> None
        | Some unifier -> Some (lazy (sld_tree world 
            (apply_subst_on_termlist unifier (right_member@other_request_terms)) 
            (unifier::subs) (n+1)))
    )) world);;

type 'a tree = Leaf of 'a | Node of 'a tree Lazy.t list


val sld_tree :
  clause list ->
  term list -> term VarMap.t list -> int -> term VarMap.t list tree = <fun>


In [16]:
let list_to_seq l = List.fold_right (fun x s -> (fun ()->Seq.Cons(x, s)) ) l Seq.empty;;

let rec to_seq = function
| Leaf str -> Seq.return str
| Node tl -> Seq.flat_map (fun par -> to_seq (Lazy.force par)) (list_to_seq tl);;

let solutions world req = let tree = sld_tree world (read_termlist req) [] 1 in 
let vars = (find_vars_in_termlist (read_termlist req)) in
Seq.map  (fun l -> vars, 
List.fold_right apply_subst_on_termlist l vars
) (to_seq tree);;

let request world req = let sol = solutions world req in
if sol () = Seq.Nil then Format.printf "This is false.\n%!"
else Seq.iter (fun (vars, tl) -> 
if vars = [] then Format.printf "This is true.\n%!"
else Format.printf "There is : %s\n%!"
(String.concat ", " (List.map2 (fun v t -> (string_of_term v) ^ " = " ^ (string_of_term t)) vars tl))
) sol;
Format.printf "\n%!";;

val list_to_seq : 'a list -> 'a Seq.t = <fun>


val to_seq : 'a tree -> 'a Seq.t = <fun>


val solutions : clause list -> string -> (term list * term list) Seq.t =
  <fun>


val request : clause list -> string -> unit = <fun>


In [17]:
type 'a ftree = FLeaf of 'a | FNode of ('a ftree) list;;

let rec force_tree (tree : 'a tree) = match tree with
| Leaf a -> FLeaf a
| Node atl -> FNode (List.map (fun t -> force_tree (Lazy.force t)) atl)

type 'a ftree = FLeaf of 'a | FNode of 'a ftree list


val force_tree : 'a tree -> 'a ftree = <fun>


# Mémoïsation

In [18]:
(* Renvoie un set de (Var Model -> Var Request) *)

let equivalent r m =
let rec equal map_opt request model = match map_opt with 
| None -> None
| Some map ->
match request, model with
| Var vr, Var vm -> (match VarMap.find_opt vm map with
    | None -> Some (VarMap.add vm vr map)
    | Some vs -> if vs = vr then Some map else None)
| Predicate (pr, lr), Predicate (pm, lm) ->
    if pr <> pm || List.length lr <> List.length lm then None
    else List.fold_left2 equal (Some map) lr lm
| Table tr, Table tm ->
    let rec aux map_opt tbl_r tbl_m = match map_opt with
    | None -> None
    | Some map -> (
    match tbl_r, tbl_m with
    | Empty, Empty -> Some map
    | TVar vr, TVar vm -> (match VarMap.find_opt vm map with
        | None -> Some (VarMap.add vm vr map)
        | Some vs -> if vs = vr then Some map else None)
    | NonEmpty (head_r, tail_r), NonEmpty (head_m, tail_m) ->
        aux (equal (Some map) head_r head_m) tail_r tail_m
    | _, _ -> None
    ) in aux (Some map) tr tm
| _, _ -> None
in equal (Some (VarMap.empty)) r m;;

let test_eq s1 s2 = Option.iter 
(VarMap.iter (fun key v -> Format.printf "%s -> %s\n%!" (string_of_term (Var key)) (string_of_term (Var v))))
(equivalent (read_term s1) (read_term s2));;

test_eq "table(A, B)" "table(C, C)";;

test_eq "table(A, A)" "table(B, C)"

val equivalent : term -> term -> var VarMap.t option = <fun>


val test_eq : string -> string -> unit = <fun>


- : unit = ()


B -> A
C -> A


- : unit = ()


# Essais de l'implémentation

## Tests basiques... des élèves, des cours et des professeurs

In [19]:
let world1 = read_program "
apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).

etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
" in

request world1 "etudiant_de(E, pierre)";

request world1 "etudiant_de(E, pierre), etudiant_de(E, alice)";

request world1 "etudiant_de(A, B)";

request world1 "etudiant_de(A, A)";

request world1 "apprend(A, A)";

request world1 "enseigne(A, A)";

request world1 "enseigne(alice, physique)";

request world1 "enseigne(alice, mathematiques)";; 

There is : E = eve
There is : E = benjamin

There is : E = benjamin

There is : A = eve, B = pierre
There is : A = benjamin, B = pierre
There is : A = benjamin, B = alice

This is false.

This is false.

This is false.

This is true.

This is false.



- : unit = ()


## Le métro londonien

In [20]:
let world = read_program "
connected(bond_street,oxford_circus,central).
connected(oxford_circus,tottenham_court_road,central).
connected(bond_street,green_park,jubilee).
connected(green_park,charing_cross,jubilee).
connected(green_park,piccadilly_circus,piccadilly).
connected(piccadilly_circus,leicester_square,piccadilly).
connected(green_park,oxford_circus,victoria).
connected(oxford_circus,piccadilly_circus,bakerloo).
connected(piccadilly_circus,charing_cross,bakerloo).
connected(tottenham_court_road,leicester_square,northern).
connected(leicester_square,charing_cross,northern).

nearby(X,Y):-connected(X,Y,L).
nearby(X,Y):-connected(X,Z,L),connected(Z,Y,L).

reachable(X,Y):-connected(X,Y,L).
reachable(X,Y):-connected(X,Z,L),reachable(Z,Y).

path(X,Y,noroute):-connected(X,Y,L).
path(X,Y,route(Z,R)):-connected(X,Z,L),path(Z,Y,R)."
in

request world "connected(piccadilly_circus,leicester_square,piccadilly)";

request world "nearby(oxford_circus, charing_cross)";

request world "nearby(tottenham_court_road,W)";

request world "reachable(bond_street, leicester_square)";

request world "connected(oxford_circus, bond_street, L)";

request world "path(oxford_circus, charing_cross, R)";;

This is true.

This is true.

There is : W = leicester_square
There is : W = charing_cross

This is true.
This is true.
This is true.
This is true.
This is true.

This is false.

There is : R = route(tottenham_court_road, route(leicester_square, noroute))
There is : R = route(piccadilly_circus, noroute)
There is : R = route(piccadilly_circus, route(leicester_square, noroute))



- : unit = ()


## Les listes

In [21]:
let world4 = read_program "
member(X, [X|Xs]).
member(X, [Y|Ys]) :- member(X, Ys).

prefix([], Ys).
prefix([X|Xs],[X|Ys]) :- prefix(Xs, Ys).

sublist(Xs, Ys) :- prefix(Xs, Ys).
sublist(Xs, [Y|Ys]) :- sublist (Xs, Ys).

append([], Ys, Ys).
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs).

reverse([],[]).
reverse([X|Xs], R) :- reverse(Xs, Z), append(Z, [X], R).

adjacent(X,Y,Zs) :- append(As, [X,Y|Ys] ,Zs).

equal(X, X).


" in

request world4 "member(a, [c,d,a,b])";
request world4 "prefix(P, [c,d,a,b])"; 
request world4 "sublist(S, [a,b,c,d])";
request world4 "append([a,b,c], [d,e,f], X)";
request world4 "reverse([a,b,c,d,e,f], R)";
request world4 "adjacent(X, Y, [a,b,c,d])";
request world4 "equal([a, b], X)";;

This is true.

There is : P = []
There is : P = [c]
There is : P = [c, d]
There is : P = [c, d, a]
There is : P = [c, d, a, b]

There is : S = []
There is : S = [a]
There is : S = [a, b]
There is : S = [a, b, c]
There is : S = [a, b, c, d]
There is : S = []
There is : S = [b]
There is : S = [b, c]
There is : S = [b, c, d]
There is : S = []
There is : S = [c]
There is : S = [c, d]
There is : S = []
There is : S = [d]
There is : S = []

There is : X = [a, b, c, d, e, f]

There is : R = [f, e, d, c, b, a]

There is : X = a, Y = b
There is : X = b, Y = c
There is : X = c, Y = d

There is : X = [a, b]



- : unit = ()


In [22]:
let world5 = read_program "
satisfiable(true).
satisfiable(and(X, Y)) :- satisfiable(X), satisfiable(Y).
satisfiable(or(X, Y)) :- satisfiable(X).
satisfiable(or(X, Y)) :- satisfiable(Y).
satisfiable(not(X)) :- invalid(X).

invalid(false).
invalid(or(X, Y)) :- invalid(X), invalid(Y).
invalid(and(X, Y)) :- invalid(X).
invalid(and(X, Y)) :- invalid(Y).
invalid(not(X)) :- satisfiable(X).
" in

request world5 "satisfiable(false)"

This is false.



- : unit = ()
